### Deepmind dm_control 분석

In [1]:
import collections

from dm_control import mujoco
from dm_control.suite import common
from dm_control.utils import containers


_DEFAULT_TIME_LIMIT = 25
_CONTROL_TIMESTEP = .025

# Minimal height of torso over foot above which stand reward is 1.
_STAND_HEIGHT = 1.2

# Horizontal speeds (meters/second) above which move reward is 1.
_WALK_SPEED = 1
_RUN_SPEED = 8


SUITE = containers.TaggedTasks()

/home/pipaek/miniconda3/envs/tdmpc2/lib/python3.9/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


1. 아래 common.read_model에 분석할 대상 xml 을 지정하시오.

    예)/home/pipaek/data/project/dm_control/dm_control/suite/walker.xml

In [2]:
def get_model_and_assets():
  """Returns a tuple containing the model XML string and a dict of assets."""
  return common.read_model('walker.xml'), common.ASSETS

action의 경우 xml 파일 안에 잘 정의되어 있음

```
<actuator>
    <motor name="right_hip" joint="right_hip" gear="100"/>
    <motor name="right_knee" joint="right_knee" gear="50"/>
    <motor name="right_ankle" joint="right_ankle" gear="20"/>
    <motor name="left_hip" joint="left_hip" gear="100"/>
    <motor name="left_knee" joint="left_knee" gear="50"/>
    <motor name="left_ankle" joint="left_ankle" gear="20"/>
</actuator>
```

2. 해당 모델의 파이썬 모듈에서 Physics 정의를 가져오시오.

    예)/home/pipaek/data/project/dm_control/dm_control/suite/walker.py

In [3]:
class Physics(mujoco.Physics):
  """Physics simulation with additional features for the Walker domain."""

  def torso_upright(self):
    """Returns projection from z-axes of torso to the z-axes of world."""
    return self.named.data.xmat['torso', 'zz']

  def torso_height(self):
    """Returns the height of the torso."""
    return self.named.data.xpos['torso', 'z']

  def horizontal_velocity(self):
    """Returns the horizontal velocity of the center-of-mass."""
    return self.named.data.sensordata['torso_subtreelinvel'][0]

  def orientations(self):
    """Returns planar orientations of all bodies."""
    return self.named.data.xmat[1:, ['xx', 'xz']].ravel()


3. task의 observation_spec과 action_spec

In [12]:
from dm_control import suite
from envs.wrappers.pixels import PixelWrapper
from envs.wrappers.tensor import TensorWrapper
from envs.dmcontrol import make_env

from omegaconf import OmegaConf
cfg = OmegaConf.load('config.yaml')
cfg.task = 'walker-walk'
env=make_env(cfg)

# env = suite.load('walker',
# 					 'walk',
# 					 task_kwargs={'random': 1},
# 					 visualize_reward=False)
# env = TensorWrapper(env)

In [14]:
env.observation_space

Box([-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf], (24,), float32)

In [15]:
env.action_space

Box([-1. -1. -1. -1. -1. -1.], [1. 1. 1. 1. 1. 1.], (6,), float32)

4. 2번에서 구했던 파이썬 모듈의 task의 클래스에서 get_observation 메소드를 찾아서 physics와의 연결고리를 구하라. 
이후 physics.named.data 와의 연결고리를 찾아나가다 보면 답을 얻을 수 있다. 

```
def get_observation(self, physics):
    """Returns an observation of body orientations, height and velocites."""
    obs = collections.OrderedDict()
    obs['orientations'] = physics.orientations()
    obs['height'] = physics.torso_height()
    obs['velocity'] = physics.velocity()
    return obs
```

In [6]:
physics = Physics.from_xml_string(*get_model_and_assets())

In [7]:
physics.orientations()

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.])

In [8]:
physics.torso_height()

1.3

In [9]:
physics.velocity()

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
physics.named.data

<dm_control.mujoco.index.make_struct_indexer.<locals>.StructIndexer at 0x7fef0dc12370>

In [12]:
physics.named.data.xpos

FieldIndexer(xpos):
                x         y         z         
0       world [ 0         0         0       ]
1       torso [ 0         0         1.3     ]
2 right_thigh [ 0        -0.05      1       ]
3   right_leg [ 0        -0.05      0.3     ]
4  right_foot [ 0.06     -0.05      0.05    ]
5  left_thigh [ 0         0.05      1       ]
6    left_leg [ 0         0.05      0.3     ]
7   left_foot [ 0.06      0.05      0.05    ]

In [13]:
physics.named.data.xmat

FieldIndexer(xmat):
                xx        xy        xz        yx        yy        yz        zx        zy        zz        
0       world [ 1         0         0         0         1         0         0
  0         1       ]
1       torso [ 1         0         0         0         1         0         0
  0         1       ]
2 right_thigh [ 1         0         0         0         1         0         0
  0         1       ]
3   right_leg [ 1         0         0         0         1         0         0
  0         1       ]
4  right_foot [ 1         0         0         0         1         0         0
  0         1       ]
5  left_thigh [ 1         0         0         0         1         0         0
  0         1       ]
6    left_leg [ 1         0         0         0         1         0         0
  0         1       ]
7   left_foot [ 1         0         0         0         1         0         0
  0         1       ]

In [16]:
physics.named.data.sensordata

FieldIndexer(sensordata):
0 torso_subtreelinvel [ 0       ]
1 torso_subtreelinvel [ 0       ]
2 torso_subtreelinvel [ 0       ]

In [15]:
physics.named.data.xmat[1:, ['xx', 'xz']]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [18]:
physics.named.data.sensordata['torso_subtreelinvel']

array([0., 0., 0.])

In [19]:
physics.velocity()

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
physics.data.qvel

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
physics.data

In [22]:
physics.data.ptr

In [24]:
physics.named.data.qvel

FieldIndexer(qvel):
0       rootz [ 0       ]
1       rootx [ 0       ]
2       rooty [ 0       ]
3   right_hip [ 0       ]
4  right_knee [ 0       ]
5 right_ankle [ 0       ]
6    left_hip [ 0       ]
7   left_knee [ 0       ]
8  left_ankle [ 0       ]